In [2]:
## -----------------------------------------------------------------------
## FOR Raw_Manual_Labelled_Dataset
## -----------------------------------------------------------------------

import re
import unicodedata
import string
import pandas as pd
from nltk.tokenize import word_tokenize
from tqdm import tqdm

tqdm.pandas()  # Enable tqdm for pandas

# Initialize the stemmer (replace with actual stemmer initialization)
# For demonstration, I'll create a dummy stemmer
class SinhalaStemmer:
    def stem(self, word):
        # Dummy stemmer: returns the word as is
        return {'stem': word}

# Define a basic English-to-Sinhala dictionary
english_to_sinhala = {
    "mental": "මානසික",
    "health": "සෞඛය",
    "issue": "ගැටළුව",
    "problem": "ගැටළුව",
    "anxiety": "කංසාව",
    "depression": "විශාදය",
    "panic": "සන්ත්රාස",
    "attack": "වාර්තාව",
    "panic attack": "සන්ත්රාස කාංසාව",
    "panic attacks": "සන්ත්රාස කාංසාව",
    "overthinking": "ඕනාවට වඩා සිතීම",
    "pulstating": "ස්පන්දනය",
    "social": "සමාජ",
    "social anxiety": "සමාජ කාංසාව",
    "busy": "කාර්යබහුල",
    "work": "වැඩ",
    "stress": "තදබදය",
    "worry": "සැලකිලිමත් වීම / වායි",
    "fear": "භීතිය",
    "insomnia": "නිදාගත නොහැකියාව",
    "sadness": "දුක්",
    "lonely": "පාලුවෙන්",
    "overwhelmed": "ඇති ආකාරයෙන් පීඩනයෙන්",
    "tired": "දුස්ස්වීම",
    "nervous": "සන්සුන් නොවන",
    "exhausted": "වියළී ගිය",
    "helpless": "අසරණ",
    "restless": "නොසන්සුන්",
    "hopeless": "බලාපොරොත්තු නැති",
    "frustrated": "හුස්ම ගන්න බැරි මට්ටමින්",
    "pressure": "පීඩනය",
    "overthink": "ඕනෑවට වඩා සිතන්න",
    "symptoms": "ලක්ෂණ",
    "burden": "බර",
    "nervousness": "ඉඟි ගැන්වීම",
    "worried": "කනස්සල්ලට පත්",
    "sensitive": "සැලකිලිමත්",
    "broken": "බිඳුණු",
    "nauseous": "නොසන්සුන්",
    "sleep": "නිද्रा",
    "lethargic": "අක්‍රිය",
    "disturbing": "ප්‍රයෝජනවත් නොවන",
    "anxious": "කනස්සල්ලට පත්",
    "calm": "සන්සුන්",
    "disorder": "රෝග",
    "strange": "අමුතු",
    "worriedness": "කනස්සල්ල",
    "worrying": "කනස්සලද",
    "tears": "කඳුළු",
    "chaos": "අව්‍යවස්ථාව",
    "feelings": "සංවේදන",
    "feeling": "සංවේදනය",
    "decision": "තීරණය",
    "decisions": "තීරණ",
    "sorrow": "දුක",
    "sorrows": "දුක්ක",
    "soul": "ආත්මය",
    "unsettled": "නොසන්සුන්",
    "dizzy": "වටකුරු",
    "social media": "සමාජ මාධ්‍ය",
    "idk": "මට නොදැනේ",
    "unclear": "අවැගහන",
    "bad": "නරක",
    "thoughts": "සිතැගි",
    "restlessness": "අශාන්තිකතාව",
    "scared": "බියගත්",
    "severe": "මහා",
    "mental": "මානසික",
    "health": "සෞඛ්‍යය",
    "pain": "වේදනාව",
    "burden ": "බර ",
    "worry": "කනස්සල්ල",
    "restless": "අශාන්තික",
    "naturally": "ස්වාභාවිකවම",
    "despair": "අපේක්ෂාව",
    "expectations": "අපේක්ෂා",
    "reason": "හේතුව",
    "survive": "ගැලවීම",
    "compromise": "සන්සුන් වීම",
    "upset": "කලබලකාරී",
    "dizziness": "වටකුරුකම",
    "tiredness": "කලබල",
    "crying": "අඳුනෑම",
    "chest": "පොටුව",
    "pain": "වේදනාව",
    "afraid": "බියගත්",
    "sudden": "හදිසි",
    "frantic": "ගැලවුණු",
    "emotional": "සංවේදී",
    "over": "අධික",
    "prevention": "ආරක්ෂාව",
    "nausea": "ඔක්කාරය",
    "doubts": "සැක",
    "illness": "ආබාධය",
    "serious": "ගැඹුරු",
    "condition": "තත්ත්වය",
    "discouraged": "අත්හරින්න",
    "losing ": "අහිමිවීම ",
    "time": "කාලය",
    "run": "දුවනවා",
    "out": "පිටතට",
    "older": "වයස්ගත",
    "bother": "හිත",
    "tight": "තද",
    "chest": "පොටුව",
    "weight": "බර",
    "loss": "අහිමිවීම",
    "excessive": "අධික",
    "hurt": "ගොඩක්",
    "stop": "නවත්වන්න",
    "let": "අවසන් කරන්න",
    "down": "පහලට",
    "exhaustion": "ශාක්තිමත්",
    "agonize": "වේදනාවට",
    "cry": "කඳුළු වන්න",
    "frustration": "අසාර්ථක බව",
    "guilt": "අපරාධ බර",
    "lonely": "තනිකඩ",
    "trigger": "සක්‍රීය කරන්න",
    "stutter": "විකාශනය",
    "extremely": "අතිශය",
    "future": "අනාගතය",
    "terrible": "දරුණු",
    "therapy": "සංවර්ධන",
    "medicine": "ඖෂධය",
    "medication": "ඖෂධ ප්‍රතිකාරය",
    "abandonment": "අත්හැරීම",
    "insecure": "අරක්ෂිත",
    "shortness": "කෙටි බව",
    "breath": "ආශ්වාසය",
    "unbearable": "අසහනය",
    "struggle": "සටන්"
}

# Additional common words and Sinhala names to remove as stopwords
additional_useless_words = [
    "මම", "මට", "මගේ", "මන්", "මං", "කරන්න", "කියලා", "එක", "වගේ", "ගොඩක්", "ඔහෙ",
    "ඔයා", "අපි", "හරි", "මෙහෙම", "එහෙම", "අද", "ඔහු", "ඇය", "ඔවුන්", "අපේ",
    "ඉතින්", "දැන්", "ඔය", "මේක", "හැබැයි", "කොහෙන්", "කොහොමද", "කොහෙද", "කොහෙත්",
    "කියනවා", "කියන", "කරපු", "කල", "අලුත්", "ඇත්ත", "වඩා", "වගේම", "එහේ", "මෙහේ",
    "තව", "ඔබ", "ඔබගේ", "නැති", "හෙට", "රැයට", "දවස්", "පිට", "වඩාත්", "ඕනෙ",
    "ඔව්", "නැහැ", "ඒක", "මේක", "ඒකද", "නැද්ද", "එහෙනම්", "කවුරුත්", "කවුරුද",
    "මොකක්ද", "මොනවද", "මොනාද", "යන්න", "දෙන්න", "හිත", "හිතලා", "හිතනවා",
    "කය", "අනිත්", "කොහොමහරි", "මොකක් වුනත්", "වාගේ", "ඔනෑම", "වෙන්න",
    "තිබෙනවා", "තිබුනා", "තිබුණා", "කරලා", "කරනවා", "කලහ", "දකින්න", "දැක්කා",
    "දක්වයි", "තෝරාගත්", "ගන්න", "එබඳු", "කෙසේද", "වර්තමානයෙ", "සෑහෙන", 
    "එහෙයින්", "බොහෝවිට", "අනිවාර්යයෙන්", "තවම", "හෝද", "නැද්ද",
    "හැඩකට", "කියවන්න", "ලැබෙයි", "දවස", "ඔබේ", "ඔබට", "එයා", "කෙනෙක්",
    "කැමතිනම්", "කියල", "එත්", "එක්ක", "දෙයක්", "වීඩියෝව", "ඊමේල්", "දුරකථන", "හරහා", "නාලිකාව", 
    "මගෙ", "එකක්", "ඇයට", "ඇඩ්මින්", "බික්ශූන්", "ත්", "මටත්", "බන්", "උබ", "ඔයාගේ"
]

# Extend original stopwords with additional common words and Sinhala names
sinhala_stopwords = [
    "සහ", "සමග", "සමඟ", "අහා", "ආහ්", "ආ", "ඕහෝ", "අනේ", "අඳෝ", "අපොයි", "අපෝ", 
    "අයියෝ", "ආයි", "ඌයි", "චී", "චිහ්", "චික්", "හෝ", "දෝ", "මෙන්", "සේ", "වැනි", 
    "බඳු", "වන්", "අයුරු", "අයුරින්", "ලෙස", "වැඩි", "ශ්‍රී", "හා", "ය", "නිසා", 
    "නිසාවෙන්", "බවට", "බව", "බවෙන්", "නම්", "සිට", "දී", "මහා", "මහ", "පමණ", 
    "පමණින්", "වන", "විට", "විටින්", "මේ", "මෙලෙස", "මෙයින්", "ඇති", "ලෙස", "සිදු", 
    "වශයෙන්", "යන", "සඳහා", "මගින්", "ඉතා", "ඒ", "එම", "ද", "අතර", "විසින්", 
    "පිළිබඳව", "පිළිබඳ", "තුළ", "වෙත", "වෙතින්", "වෙතට", "වෙනුවෙන්", "වෙනුවට", 
    "වෙන", "ගැන", "නෑ", "අනුව", "පරිදි", "තෙක්", "මෙතෙක්", "මේතාක්", "තුරු", "තුරා", "තුරාවට", "තුලින්", "නමුත්", 
    "එනමුත්", "වස්", "මෙන්", "පරිදි", "සෑම", "ඔබ", "අවශ්ය", "ඔබගේ", "වුණා", 
    "කෙතරම්", "කෙසේ", "එක්", "සමග", "තවත්"
]
sinhala_stopwords.extend(additional_useless_words)

stemmer = SinhalaStemmer()

def unicode_normalize(text):
    return unicodedata.normalize('NFC', text)

def remove_urls(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def remove_mentions(text):
    return re.sub(r'@\w+', '', text)

def remove_digits(text):
    return re.sub(r'\d+', '', text)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F6FF"  # Emoticons
        u"\U0001F1E0-\U0001F1FF"  # Flags
        u"\U0001FA70-\U0001FAFF"  # Symbols & Pictographs Extended-A
        u"\u200d"                 # Zero Width Joiner
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()

def contains_repeated_phrases(text):
    words = text.split()
    for i in range(len(words) - 3):
        if words[i] == words[i+1] == words[i+2]:
            return True
    return False

def remove_repeated_phrases(text):
    words = text.split()
    filtered_words = []
    i = 0
    while i < len(words):
        if i + 2 < len(words) and words[i] == words[i+1] == words[i+2]:
            repeated_word = words[i]
            filtered_words.append(repeated_word)
            j = i + 3
            while j < len(words) and words[j] == repeated_word:
                j += 1
            i = j
        else:
            filtered_words.append(words[i])
            i += 1
    return ' '.join(filtered_words)

def to_lowercase(text):
    return text.lower()

def is_english_word(word):
    return re.match(r'^[A-Za-z]+$', word) is not None

def translate_english_to_sinhala(text):
    words = text.split()
    translated_words = []
    for w in words:
        if is_english_word(w):
            # Convert to lowercase to match dictionary keys
            w_lower = w.lower()
            if w_lower in english_to_sinhala:
                translated_words.append(english_to_sinhala[w_lower])
            else:
                translated_words.append(w)  # Retain if not in dictionary
        else:
            translated_words.append(w)
    return ' '.join(translated_words)

def basic_tokenize(text):
    return word_tokenize(text)

def remove_stopwords(tokens, stopwords):
    return [t for t in tokens if t not in stopwords]

def remove_short_tokens(tokens, min_length=2):
    return [t for t in tokens if len(t) >= min_length]

def sinhala_stem(word):
    stem_result = stemmer.stem(word)
    if isinstance(stem_result, dict) and 'stem' in stem_result:
        return stem_result['stem']
    return word

def stem_tokens(tokens):
    return [sinhala_stem(t) for t in tokens]

def clean_sinhala_token(token):
    token = token.replace('\u200d', '')
    return token.strip()

def filter_non_sinhala_tokens(tokens):
    sinhala_pattern = re.compile(r'^[\u0D80-\u0DFF]+$')  # Sinhala Unicode range
    filtered = []
    for t in tokens:
        t = clean_sinhala_token(t)
        if sinhala_pattern.match(t):
            filtered.append(t)
    return filtered

def preprocess_text(text):
    # Step 1: Unicode normalization
    text = unicode_normalize(text)
    
    # Step 2: Remove URLs
    text = remove_urls(text)
    
    # Step 3: Remove mentions
    text = remove_mentions(text)
    
    # Step 4: Remove emojis
    text = remove_emojis(text)
    
    # Step 5: Remove digits
    text = remove_digits(text)
    
    # Step 6: Remove punctuation
    text = remove_punctuation(text)
    
    # Step 7: Convert to lowercase
    text = to_lowercase(text)
    
    # Step 8: Remove extra spaces
    text = remove_extra_spaces(text)

    # Step 9: Remove repeated phrases if any
    if contains_repeated_phrases(text):
        text = remove_repeated_phrases(text)

    # Step 10: Translate English words to Sinhala
    text = translate_english_to_sinhala(text)

    # Step 11: Tokenize the text
    tokens = basic_tokenize(text)
    
    # Step 12: Remove stopwords
    tokens = remove_stopwords(tokens, sinhala_stopwords)
    
    # Step 13: Remove short tokens
    tokens = remove_short_tokens(tokens, min_length=2)
    
    # Step 14: Filter non-Sinhala tokens
    tokens = filter_non_sinhala_tokens(tokens)
    
    # Step 15: Stem tokens
    tokens = stem_tokens(tokens)
    
    # Final cleaned text
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text, tokens

if __name__ == "__main__":
    
    # -------------------------------
    # Loading the Dataset
    # -------------------------------
    
    # Define the input and output file paths
    input_csv = "../Data/raw_Dataset.csv"
    output_csv = "../Preprocessed_Manual_Labelled_Dataset.csv"
    
    try:
        # Load the raw, unlabelled dataset
        df = pd.read_csv(input_csv)
        print(f"Loaded '{input_csv}' successfully.")
    except FileNotFoundError:
        print(f"Error: '{input_csv}' not found. Please check the file path.")
        exit(1)
    except Exception as e:
        print(f"Error loading '{input_csv}': {e}")
        exit(1)
    
    # Check if 'raw_text' column exists
    if 'raw_text' not in df.columns:
        print("Error: 'raw_text' column not found in the input CSV.")
        exit(1)
    
    # -------------------------------
    # Preprocessing the Text Data
    # -------------------------------
    
    print("Starting text preprocessing...")
    df['cleaned_text'], df['cleaned_tokens'] = zip(*df['raw_text'].progress_apply(preprocess_text))
    print("Text preprocessing completed.")
    
    # -------------------------------
    # Removing Duplicate Texts
    # -------------------------------
    
    print("Removing duplicate texts based on 'cleaned_text' column...")
    
    # Before removing duplicates
    initial_count = df.shape[0]
    
    # Drop duplicate 'cleaned_text' entries, keeping the first occurrence
    df = df.drop_duplicates(subset=['cleaned_text'], keep='first').reset_index(drop=True)
    
    # After removing duplicates
    final_count = df.shape[0]
    duplicates_removed = initial_count - final_count
    
    print(f"Removed {duplicates_removed} duplicate entries. Dataset size reduced from {initial_count} to {final_count}.")
    
    # -------------------------------
    # Saving the Preprocessed Dataset
    # -------------------------------
    
    try:
        df.to_csv(output_csv, index=False)
        print(f"Preprocessed and deduplicated dataset saved to '{output_csv}'.")
    except Exception as e:
        print(f"Error saving preprocessed dataset: {e}")

Loaded '../Data/raw_Dataset.csv' successfully.
Starting text preprocessing...


100%|██████████| 2653/2653 [00:00<00:00, 3209.72it/s]

Text preprocessing completed.
Removing duplicate texts based on 'cleaned_text' column...
Removed 33 duplicate entries. Dataset size reduced from 2653 to 2620.
Preprocessed and deduplicated dataset saved to '../Preprocessed_Manual_Labelled_Dataset.csv'.
